In [ ]:
from pathlib import Path

import numpy as np
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio

from src.data.monuseg import get_dataset, tf_random_rotate, tf_random_crop


In [ ]:
ds = get_dataset().map(lambda x, y: tf_random_crop(x, y, rotation=True))


In [ ]:
image, mask = next(ds.as_numpy_iterator())

In [ ]:
mask.shape

In [ ]:
plt.imshow(image)

In [ ]:
plt.imshow(mask)

In [ ]:
mask_files = [f for f in Path("/home/valentin/python_wkspce/2d_bispectrum_cnn/data/raw/MoNuSeg2018Training/Masks/binary").rglob("*.tiff")]

In [ ]:
mask_shape_x = list()
mask_shape_y = list()
for f in mask_files:
    mask = np.array(Image.open(str(f)) )
    mask_shape_x.append(mask.shape[0])
    mask_shape_y.append(mask.shape[1])

In [ ]:
np.unique(mask_shape_y)